In [1]:
import numpy as np
import pandas as pd
from numpy_financial import irr

# Feasibility Analysis of a wind pumped hydroelectric storage system on the Island of Brava

# Load data
VentoCurvaPotencia = np.loadtxt('VentoCurvaPotencia.txt')
#Pd = pd.read_excel('Brava_consumo.xlsx', skiprows=2, usecols=range(8, 28))  # Assuming the data starts from row 3
# Carregue o arquivo Excel
xl = pd.ExcelFile('Brava_consumo.xlsx')
#Pd = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')
# Leia os dados da aba "Carga2010-2037"
Pd = pd.read_excel(xl, sheet_name='Carga2010-2037', dtype=float,nrows=8762)
# Wind Turbine Groups: [Power, Quantity, Corresponding column, Reservoir volume]

Cen = np.zeros((7, 4))  # inicializacao da matriz de cenarios
Cen[0, :] = [150, 2, 2, 1500]
Cen[1, :] = [300, 1, 3, 1500]

Cen[2, :] = [150, 3, 2, 2250]
Cen[3, :] = [225, 2, 4, 2250]

Cen[4, :] = [225, 3, 4, 3400]
Cen[5, :] = [300, 2, 3, 3000]
Cen[6, :] = [300, 3, 3, 4500]

# 1a-potencia do aerogerador;
# 2a-quantidade de aerogeradores; 
# 3a-especifica a coluna onde vai pegar a potencia do aerogerador correspondente; 2(150kW), 3(225kW) e 4(300kW)
# 4a-volume do reservatório em m3

Res = np.zeros((Cen.shape[0], 14))  # inicializacao da matriz dos resultados dos n cenários 
Res2 = np.zeros((Cen.shape[0], 8))  # inicializacao da matriz dos resultados dos n cenários


# Loop principal para cada reservatório definido em Cen
for k in range(len(Cen)):

    ano = 20  # Número de anos do projeto
    RS = Cen[k, 3]  # Capacidade do reservatório em m³
    ERmax = RS / 3600 * 9.8 * 110  # Capacidade do reservatório em kWh
    h = 8760  # Número de horas em um ano

    # Inicialização das matrizes para os valores anuais e horários
    Er = np.zeros((h, ano))  # Energia do reservatório
    Erj = ERmax / 2  # Energia inicial no reservatório
    Pt = np.zeros((h, ano))  # Potência da turbina hidráulica
    Pb = np.zeros((h, ano))  # Energia usada para bombeamento
    Erej = np.zeros((h, ano))  # Energia rejeitada
    Pge = np.zeros((h, ano))  # Potência do gerador diesel
    Pwr = np.zeros((h, ano))  # Potência eólica injetada na rede

    # Eficiências
    nb = 0.86  # Eficiência média das bombas
    nt = 0.86  # Eficiência média das turbinas hidráulicas
    nw = 0.97  # Eficiência elétrica dos aerogeradores

    Te = Cen[k, 0]  # Potência unitária do aerogerador
    ne = Cen[k, 1]  # Número de aerogeradores
    PDmin = 256 * 0.5  # Potência mínima do gerador diesel (50% da capacidade)

    # Potência eólica disponível por hora com base na curva de potência
    Pwt = ne * nw * VentoCurvaPotencia(np.arange(h), Cen[k, 2])

    Ptmax = np.mean(Pwt)  # Potência nominal da turbina hidráulica

    # Iteração para cada ano
    for i in range(ano):

        # Iteração para cada hora
        for j in range(h):

            # PRODUÇÃO EÓLICA E BOMBEAMENTO
            if Pwt[j] > Pd[j, i] * 0.35:  # Excedente eólico

                Pwr[j, i] = Pd[j, i] * 0.35  # Máximo injetado na rede

                # Aproveitar o excedente para bombeamento
                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):
                    Pb[j, i] = Pwt[j] - Pwr[j, i]  # Energia usada no bombeamento
                    Erj += (Pwt[j] - Pwr[j, i]) * nb  # Atualiza nível do reservatório
                else:
                    Pb[j, i] = (ERmax - Erj) / nb  # Energia limitada pela capacidade
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb  # Energia rejeitada
                    Erj = ERmax  # Reservatório cheio

            else:  # Energia eólica abaixo do limite
                Pwr[j, i] = Pwt[j]

            # ENERGIA DE COMPENSAÇÃO (HIDRÁULICA OU DIESEL)
            deficit = Pd[j, i] - Pwr[j, i]

            if deficit <= Erj * nt:  # Energia hídrica suficiente
                if deficit <= Ptmax:
                    Pt[j, i] = deficit
                    Erj -= Pt[j, i] / nt
                elif Ptmax < deficit <= (Ptmax + PDmin):
                    Pge[j, i] = PDmin
                    Pt[j, i] = deficit - Pge[j, i]
                    Erj -= Pt[j, i] / nt
                else:
                    Pt[j, i] = Ptmax
                    Pge[j, i] = deficit - Pt[j, i]
                    Erj -= Ptmax / nt

            else:  # Energia hídrica insuficiente
                if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
                    if Ptmax >= PDmin:
                        Pge[j, i] = PDmin
                        Pt[j, i] = deficit - Pge[j, i]
                        Erj -= Pt[j, i] / nt
                    else:
                        Pge[j, i] = deficit
                elif Erj * nt > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = deficit - Pt[j, i]
                    Erj -= Ptmax / nt
                else:
                    Pge[j, i] = deficit

            # Atualiza energia do reservatório
            Er[j, i] = Erj


# pW = percentual de eólica na rede
pW = np.sum(Pwr) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia eolica na rede = %3.2f' % pW)

# pH = percentual de energia hídrica na rede elétrica
pH = np.sum(Pt) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia hidrica na rede = %3.2f' % pH)

# pER = percentual de penetração total de Renováveis (eólica+hidro)
pER = (np.sum(Pwr + Pt) / np.sum(Pd.to_numpy())) * 100
print('Total de RE = %3.2f' % pER)

# ERt = energia renovável aproveitada (eolica injetada diretamente e hidraulica), kWh
ERt = np.sum(Pwr + Pt)

# pDG = percentual de energia fossil na rede
pDG = np.sum(Pge) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia fossil na rede = %3.2f' % pDG)

# pdiss = percentual de energia dissipada devido a reservatorio cheio
pdiss = np.sum(Erej) / (np.sum(Pwt) * ano) * 100
print('Percentual de  Energia dissipada devido a reservatorio cheio: %3.2f' % pdiss)






TypeError: 'numpy.ndarray' object is not callable